In [2]:
import sklearn
import pandas as pd
import numpy as np
import mglearn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from IPython.display import display

import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

In [3]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [4]:
train_id = train.Id
test_id = test.Id
# Id 삭제 
train.drop('Id', axis = 1, inplace=True)
test.drop('Id', axis = 1, inplace=True)

print('train_data shape', train.shape)
print('test_data shape', test.shape)

train_data shape (15120, 55)
test_data shape (565892, 54)


In [6]:
train.tail()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
15115,2607,243,23,258,7,660,170,251,214,1282,...,0,0,0,0,0,0,0,0,0,3
15116,2603,121,19,633,195,618,249,221,91,1325,...,0,0,0,0,0,0,0,0,0,3
15117,2492,134,25,365,117,335,250,220,83,1187,...,0,0,0,0,0,0,0,0,0,3
15118,2487,167,28,218,101,242,229,237,119,932,...,0,0,0,0,0,0,0,0,0,3
15119,2475,197,34,319,78,270,189,244,164,914,...,0,0,0,0,0,0,0,0,0,3


## 변수 설명
- Elevation - Elevation in meters (높이 /meter 단위)
- Aspect - Aspect in degrees azimuth (방위각)
- Slope - Slope in degrees (기울기 각도 /도 단위)
- Horizontal_Distance_To_Hydrology - Horz Dist to nearest surface water features (수원과의 수평거리)
- Vertical_Distance_To_Hydrology - Vert Dist to nearest surface water features (수원과의 수직거리)
- Horizontal_Distance_To_Roadways - Horz Dist to nearest roadway (길가와의 수평거리)
- Hillshade_9am (0 to 255 index) - Hillshade index at 9am, summer solstice (오전 9시의 차양 / 0~255)
- Hillshade_Noon (0 to 255 index) - Hillshade index at noon, summer solstice (정오시의 차양/ 0~255)
- Hillshade_3pm (0 to 255 index) - Hillshade index at 3pm, summer solstice (오후 9시의 차양/ 0~255)
- Horizontal_Distance_To_Fire_Points - Horz Dist to nearest wildfire ignition points (야생 산불 발화지점과의 수평거리)
- Wilderness_Area (4 binary columns, 0 = absence or 1 = presence) - Wilderness area designation (황야 지대 /4종류 ) in Roosevelt National Forest of northern Colorado
- Soil_Type (40 binary columns, 0 = absence or 1 = presence) - Soil Type designation (토양 종류 / 40종류)

- 토양종류와 황야 지대 카테고리별 설명은 https://www.kaggle.com/c/forest-cover-type-prediction/data 참조


## 종속 변수
- Cover_Type (7 types, integers 1 to 7) - Forest Cover Type designation- (산림 유형 / 7종류) (the predominant kind of tree cover)
- 1 - Spruce/Fir
- 2 - Lodgepole Pine
- 3 - Ponderosa Pine
- 4 - Cottonwood/Willow
- 5 - Aspen
- 6 - Douglas-fir
- 7 - Krummholz

In [7]:
train_data = train.iloc[:, :-1]
train_target = train.iloc[:, [-1]]

all_data = pd.concat([train_data, test])
print('all_data shape', all_data.shape)

all_data shape (581012, 54)


In [49]:
train_target - 1

,Cover_Type
0,4
1,4
2,1
3,1
4,4
5,1
6,4
7,4
8,4
9,4


In [53]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, RobustScaler
import tensorflow as tf
scaler = RobustScaler()
encoder = OneHotEncoder()
train_target_onehot = encoder.fit_transform(train_target).toarray()

X_train, X_test, y_train, y_test = train_test_split(train_data, train_target_onehot, random_state = 42)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [59]:
nb_classes = 7
X = tf.placeholder(tf.float32, [None, X_train.shape[1]])
Y = tf.placeholder(tf.float32, [None, nb_classes])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.get_variable("WWi111", shape =[X_train.shape[1], 200], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([200]), name = 'bias1')
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("WWi222", shape =[200, 300], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([300]), name = 'bias2')
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("WWi333", shape =[300, 200], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([200]), name = 'bias3')
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.get_variable("WWi444", shape =[200, nb_classes], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([nb_classes]), name = 'bias4')
hypothesis = tf.matmul(L3, W4) + b4

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = hypothesis, labels = Y))

optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)
is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))

accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(2000):
        
        c , _ = sess.run([cost, optimizer], feed_dict = {X: X_train, Y: y_train, keep_prob: 0.7})
        if step  % 100 == 0:
            print('step : {}, cost = {}'.format(step, c))
    print('train set : ', sess.run(accuracy, feed_dict = {X: X_train, Y: y_train}))
    acc = sess.run(accuracy, feed_dict = {X: X_test, Y: y_test})    
    print('test set : ', acc)

step : 0, cost = 3.5122392177581787
step : 100, cost = 0.9042738676071167
step : 200, cost = 0.7047073245048523
step : 300, cost = 0.6181574463844299
step : 400, cost = 0.5662969946861267
step : 500, cost = 0.531852126121521
step : 600, cost = 0.5027167797088623
step : 700, cost = 0.4779565632343292
step : 800, cost = 0.46029701828956604
step : 900, cost = 0.4413899779319763
step : 1000, cost = 0.4270518720149994
step : 1100, cost = 0.406904399394989
step : 1200, cost = 0.40159139037132263
step : 1300, cost = 0.38815832138061523
step : 1400, cost = 0.3731262683868408
step : 1500, cost = 0.3583845794200897
step : 1600, cost = 0.3500693738460541
step : 1700, cost = 0.3351978063583374
step : 1800, cost = 0.3315659761428833
step : 1900, cost = 0.31938353180885315


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_18' with dtype float
	 [[Node: Placeholder_18 = Placeholder[dtype=DT_FLOAT, shape=<unknown>, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Placeholder_18', defined at:
  File "C:\Users\kjw1o\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\kjw1o\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\kjw1o\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\kjw1o\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\kjw1o\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\kjw1o\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File

In [56]:
nb_classes = 7
X = tf.placeholder(tf.float32, [None, X_train.shape[1]])
Y = tf.placeholder(tf.float32, [None, nb_classes])

W1 = tf.Variable(tf.random_normal([X_train.shape[1], 200]), name = 'weight1')
b1 = tf.Variable(tf.random_normal([200]), name = 'bias1')
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([200, 300]), name = 'weight2')
b2 = tf.Variable(tf.random_normal([300]), name = 'bias2')
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.Variable(tf.random_normal([300, 200]), name = 'weight3')
b3 = tf.Variable(tf.random_normal([200]), name = 'bias3')
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

W4 = tf.Variable(tf.random_normal([200, nb_classes]), name = 'weight4')
b4 = tf.Variable(tf.random_normal([nb_classes]), name = 'bias4')
hypothesis = tf.matmul(L3, W4) + b4

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = hypothesis, labels = Y))

optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)
is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))

accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(5000):
        
        c , _ = sess.run([cost, optimizer], feed_dict = {X: X_train, Y: y_train})
        if step  % 100 == 0:
            print('step : {}, cost = {}'.format(step, c))
    print('train set : ', sess.run(accuracy, feed_dict = {X: X_train, Y: y_train}))
    acc = sess.run(accuracy, feed_dict = {X: X_test, Y: y_test})    
    print('test set : ', acc)

step : 0, cost = 4739.9931640625
step : 100, cost = 275.20916748046875
step : 200, cost = 124.11548614501953
step : 300, cost = 69.53717041015625
step : 400, cost = 40.854515075683594
step : 500, cost = 25.224870681762695
step : 600, cost = 16.252470016479492
step : 700, cost = 10.626121520996094
step : 800, cost = 6.960610389709473
step : 900, cost = 4.687759876251221
step : 1000, cost = 3.1042065620422363
step : 1100, cost = 1.7704102993011475
step : 1200, cost = 1.2081860303878784
step : 1300, cost = 0.6732842326164246
step : 1400, cost = 0.3605567216873169
step : 1500, cost = 0.21598337590694427
step : 1600, cost = 0.12086567282676697
step : 1700, cost = 0.06732814013957977
step : 1800, cost = 0.020026739686727524
step : 1900, cost = 0.016215506941080093
step : 2000, cost = 0.012787341140210629
step : 2100, cost = 0.0092325359582901
step : 2200, cost = 0.005894696805626154
step : 2300, cost = 0.002940583974123001
step : 2400, cost = 0.0019343073945492506
step : 2500, cost = 0.00166